In [156]:
from langchain.agents import initialize_agent, Tool
from langchain.agents.agent_types import AgentType
from langchain_openai import ChatOpenAI
from langchain.tools import tool
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.stats import norm
import joblib
from prophet import Prophet
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime, timedelta

from IPython.display import Markdown, display
import pickle
import os
import json
import warnings

warnings.filterwarnings("ignore")

In [ ]:
OPENAI_API_KEY = ""
LLM_MODEL_NAME = 'gpt-4o-mini'

In [143]:
RAW_DATA_PATH = "/home/nuh/PycharmProjects/StockAgent/data"
DATA_PATH = "/home/nuh/PycharmProjects/StockAgent/dev_v_04/data"
MODEL_PATH = "/home/nuh/PycharmProjects/StockAgent/dev_v_04/models"

# Agent Tools

## Propmt Parametreleri

In [174]:
parameters = {
    "product_id": 7,
    "mevcut_stok": 22,
    "teslim_suresi": 7,
    "siparis_maliyeti": 50,
    "stok_tutma_maliyeti": 5,
    "servis_seviyesi": 0.95,
    "stockout_cost": 20,
    "start_date": "2025-05-01",
    "end_date": "2025-05-07"
}

## Propmt Paramtreler

# Prophet Forecast Tool

In [145]:
def forecast_prophet_total(model_path: str, product_df_path: str, product_id: int, start_date: str,
                           end_date: str) -> pd.DataFrame:
    """
    Prophet ile varyant bazlı forecast. Her varyant için tek model kullanır.
    Sadece gelecekteki zaman ve varyant bilgileri gerekir.
    """
    # 1. Prophet modelini yükle
    model = joblib.load(model_path)

    # 2. Ürün varyantlarını yükle
    product_df = pd.read_csv(product_df_path)
    df = product_df[product_df["product_id"] == product_id].copy()

    # 3. Gelecekteki tarihleri oluştur
    future_dates = pd.date_range(start=start_date, end=end_date)
    forecast_rows = []

    for date in future_dates:
        for _, row in df.iterrows():
            forecast_rows.append({
                "ds": date,
                "product_id": product_id,
                "color": row["color"],
                "size": row["size"],
                "is_campaign": 0,
                "discount": 0.0
            })

    future_df = pd.DataFrame(forecast_rows)

    # 4. Prophet tahmini
    preds = model.predict(future_df[["ds", "is_campaign", "discount"]])
    future_df["y_pred"] = preds["yhat"]
    future_df["model"] = "Prophet"

    return future_df[["ds", "product_id", "color", "size", "y_pred", "model"]]


# XGBoost Forecast Tool

In [146]:
def forecast_xgboost_total(product_id: int, model_path: str, product_df_path: str, start_date: str,
                           end_date: str) -> pd.DataFrame:
    """
    Gelecekteki belirli bir tarih aralığı için XGBoost ile varyant bazlı tahmin üretir.
    Eğitimde kullanılan tüm feature'ları içerir.
    """

    # 1. Modeli yükle
    model = joblib.load(model_path)

    # 2. Ürün varyantlarını yükle
    product_df = pd.read_csv(product_df_path)
    df = product_df[product_df["product_id"] == product_id].copy()

    # Gereken alanlar varsa ekle
    df["channel"] = "Toplam"
    df["type"] = "Standart"

    # Kategorik encoding
    for col in ["color", "size", "channel", "category", "subcategory", "type"]:
        df[col] = df[col].astype("category").cat.codes

    # 3. Geleceğe ait tarih aralığını oluştur
    forecast_rows = []
    dates = pd.date_range(start=start_date, end=end_date)

    for date in dates:
        dow = date.dayofweek
        month = date.month
        week = date.isocalendar().week

        for _, row in df.iterrows():
            forecast_rows.append({
                "color": row["color"],
                "size": row["size"],
                "channel": row["channel"],
                "category": row["category"],
                "subcategory": row["subcategory"],
                "type": row["type"],
                "is_campaign": 0,
                "discount": 0.0,
                "dayofweek_sin": np.sin(2 * np.pi * dow / 7),
                "dayofweek_cos": np.cos(2 * np.pi * dow / 7),
                "month_sin": np.sin(2 * np.pi * month / 12),
                "month_cos": np.cos(2 * np.pi * month / 12),
                "week_sin": np.sin(2 * np.pi * week / 52),
                "week_cos": np.cos(2 * np.pi * week / 52),
                "ds": date,
                "product_id": product_id,
                "color_str": row["color"],  # optional: for output
                "size_str": row["size"]
            })

    input_df = pd.DataFrame(forecast_rows)

    # 4. Tahmin yap
    X = input_df[[
        "color", "size", "channel", "category", "subcategory", "type",
        "is_campaign", "discount",
        "dayofweek_sin", "dayofweek_cos", "month_sin", "month_cos", "week_sin", "week_cos"
    ]]

    input_df["y_pred"] = model.predict(X)
    input_df["model"] = "XGBoost"

    return input_df[["ds", "product_id", "color_str", "size_str", "y_pred", "model"]].rename(
        columns={"color_str": "color", "size_str": "size"}
    )


# Stok Hesaplayıcı

In [164]:
def calculate_stock_values(df, servis_seviyesi, teslim_suresi, mevcut_stok, siparis_maliyeti, stok_tutma_maliyeti):
    preds = np.array(df["y_pred"])
    mean = np.mean(preds)
    std = np.std(preds)
    z = norm.ppf(servis_seviyesi)
    SS = z * std * np.sqrt(teslim_suresi)
    ROP = mean * teslim_suresi + SS
    EOQ = np.sqrt((2 * np.sum(preds) * 12 * siparis_maliyeti) / stok_tutma_maliyeti)
    order_qty = max(ROP - mevcut_stok, 0)

    return {
        "mean": mean,
        "std": std,
        "SS": SS,
        "ROP": ROP,
        "EOQ": EOQ,
        "SiparisMiktari": order_qty
    }


In [165]:
def calculate_total_cost(ROP, EOQ, mevcut_stok, stok_tutma_maliyeti, siparis_maliyeti, stockout_cost):
    """
    ROP ve EOQ değerlerine göre toplam maliyet hesabı yapar.
    - Shortage Cost: Stoksuz kalma miktarı * birim ceza
    - Holding Cost: Ortalama stok seviyesi * stok tutma maliyeti
    - Ordering Cost: Sipariş başına sabit maliyet
    """
    shortage_qty = max(0, ROP - mevcut_stok)
    shortage_cost = stockout_cost * shortage_qty
    holding_cost = (EOQ / 2) * stok_tutma_maliyeti
    total_cost = shortage_cost + holding_cost + siparis_maliyeti

    return {
        "shortage_qty": shortage_qty,
        "shortage_cost": shortage_cost,
        "holding_cost": holding_cost,
        "ordering_cost": siparis_maliyeti,
        "total_cost": total_cost
    }



In [166]:
def calculate_variant_risk_scores(df_forecast: pd.DataFrame, mevcut_stok: int, teslim_suresi: int):
    results = []
    grouped = df_forecast.groupby(["color", "size"])
    for (color, size), group in grouped:
        mean = group["y_pred"].mean()
        std = group["y_pred"].std()
        z = norm.ppf(0.95)
        ss = z * std * np.sqrt(teslim_suresi)
        rop = mean * teslim_suresi + ss
        shortage_risk = max(0, rop - mevcut_stok)

        max_mean = df_forecast["y_pred"].max()
        max_std = df_forecast["y_pred"].std()
        normalized_mean = mean / max_mean if max_mean > 0 else 0
        normalized_std = std / max_std if max_std > 0 else 0
        normalized_shortage = shortage_risk / rop if rop > 0 else 0

        risk_score = 0.4 * normalized_mean + 0.3 * normalized_std + 0.3 * normalized_shortage
        risk_level = (
            "Yüksek Risk" if risk_score > 0.7
            else "Orta Risk" if risk_score > 0.3
            else "Düşük Risk"
        )

        results.append({
            "color": color,
            "size": size,
            "mean": mean,
            "std": std,
            "rop": rop,
            "shortage_risk": shortage_risk,
            "risk_score": round(risk_score, 3),
            "risk_level": risk_level
        })
    return pd.DataFrame(results)


In [148]:
def show_agent_output_markdown(output_text: str):
    """
    LangChain agent çıktısını markdown olarak hücreye bastırır.
    """
    display(Markdown(output_text))

## Agent Tools

In [167]:
df_prophet_forecast = forecast_prophet_total(
    model_path=f"{MODEL_PATH}/prophet/prophet_model_product_{parameters["product_id"]}.pkl",
    product_df_path=f"{DATA_PATH}/products_with_variants.csv",
    product_id=parameters["product_id"],
    start_date=parameters["start_date"],
    end_date=parameters["end_date"]
)

df_prophet_forecast_dict = df_prophet_forecast.to_dict(orient="records")

In [168]:
df_xgboost_forecast = forecast_xgboost_total(
    product_id=parameters["product_id"],
    model_path=f"{MODEL_PATH}/sklearn/xgboost_model_product_{parameters["product_id"]}.pkl",
    product_df_path=f"{DATA_PATH}/products_with_variants.csv",
    start_date=parameters["start_date"],
    end_date=parameters["end_date"]
)

df_xgboost_forecast_dict = df_xgboost_forecast.to_dict(orient="records")


In [177]:
stock_calculation_prophet = calculate_stock_values(df_prophet_forecast, parameters["servis_seviyesi"],
                                                   parameters["teslim_suresi"],
                                                   parameters["mevcut_stok"],
                                                   parameters["siparis_maliyeti"],
                                                   parameters["stok_tutma_maliyeti"])
stock_calculation_xgboost = calculate_stock_values(df_xgboost_forecast, parameters["servis_seviyesi"],
                                                   parameters["teslim_suresi"],
                                                   parameters["mevcut_stok"],
                                                   parameters["siparis_maliyeti"],
                                                   parameters["stok_tutma_maliyeti"])

variant_risk_prophet_df = calculate_variant_risk_scores(df_prophet_forecast, parameters["mevcut_stok"],
                                                        parameters["teslim_suresi"])
variant_risk_xgb_df = calculate_variant_risk_scores(df_xgboost_forecast, parameters["mevcut_stok"],
                                                    parameters["teslim_suresi"])



In [176]:
costs_prophet = calculate_total_cost(
    ROP=stock_calculation_prophet["ROP"],
    EOQ=stock_calculation_prophet["EOQ"],
    mevcut_stok=parameters["mevcut_stok"],
    stok_tutma_maliyeti=parameters["stok_tutma_maliyeti"],
    siparis_maliyeti=parameters["siparis_maliyeti"],
    stockout_cost=parameters["stockout_cost"]
)

costs_xgboost = calculate_total_cost(
    ROP=stock_calculation_xgboost["ROP"],
    EOQ=stock_calculation_xgboost["EOQ"],
    mevcut_stok=parameters["mevcut_stok"],
    stok_tutma_maliyeti=parameters["stok_tutma_maliyeti"],
    siparis_maliyeti=parameters["siparis_maliyeti"],
    stockout_cost=parameters["stockout_cost"]
)


# LLM Nihayi Sipariş Kararı

In [152]:
llm = ChatOpenAI(
    model_name=LLM_MODEL_NAME,
    temperature=0,
    openai_api_key=OPENAI_API_KEY,
)

In [184]:
prompt= f"""
Ürün ID: {parameters["product_id"]}
Tahmin edilen tarih aralığı: {parameters["start_date"]} - {parameters["end_date"]}

İşletme Parametreleri:
- Mevcut stok: {parameters["mevcut_stok"]}
- Teslim süresi: {parameters["teslim_suresi"]} gün
- Servis seviyesi: %{parameters["servis_seviyesi"] * 100}
- Sipariş maliyeti: {parameters["siparis_maliyeti"]} TL
- Stok tutma maliyeti: {parameters["stok_tutma_maliyeti"]} TL
- Stoksuz kalma maliyeti (stockout cost): {parameters["stockout_cost"]} TL

Prophet modeli çıktısı:
Model Tahminleri: {df_prophet_forecast_dict}
Stok Hesaplamaları: {stock_calculation_prophet}
Varyant Risk Skorları: {variant_risk_prophet_df.to_dict(orient='records')}
Toplam Maliyet Analizi: {costs_prophet}

XGBoost modeli çıktısı:
Model Tahminleri: {df_xgboost_forecast_dict}
Stok Hesaplamaları: {stock_calculation_xgboost}
Varyant Risk Skorları: {variant_risk_xgb_df.to_dict(orient='records')}
Toplam Maliyet Analizi: {costs_xgboost}

Lütfen aşağıdaki konuları analiz et:

1. Prophet ve XGBoost modellerinin genel tahmin ortalaması ve varyansı nedir? Hangi model daha istikrarlı ve güvenilir?
2. Her modelin EOQ, ROP ve SS değerlerini karşılaştır. Hangisi daha uygun sipariş stratejisi sunuyor?
3. Toplam maliyet analizine göre hangi model işletmeye daha az maliyet çıkarıyor? (Stok tutma, sipariş ve stoksuz kalma maliyetleri dahil)
4. Varyant bazlı risk skorlarını incele. Yüksek riskli varyantlar hangileri ve nasıl önceliklendirilmelidir?
5. Sipariş önerisi:
   - Toplam kaç adet sipariş verilmelidir?
   - Hangi varyantlara öncelik verilmeli? (Risk skorlarına göre grupla)
6. Tüm analizleri göz önünde bulundurarak nihai model seçimini yap.
7. Kararını **açık ve gerekçeli** şekilde sun:
   - Servis seviyesi, stok-out riski, maliyetler, varyant dengesi ve operasyonel uygulanabilirlik açısından değerlendir.
   - Nihai sipariş miktarını belirt ve işletmeye önerini ilet.
"""


In [185]:
response = llm.invoke(prompt)


In [186]:
output_text = response.content

show_agent_output_markdown(output_text=output_text)

### 1. Model Tahmin Ortalaması ve Varyansı
**Prophet Modeli:**
- Ortalama Tahmin: 9.59
- Varyans: 1.56

**XGBoost Modeli:**
- Ortalama Tahmin: 11.65
- Varyans: 2.63

**Analiz:**
- Prophet modeli daha düşük bir ortalama tahmin sunarken, XGBoost modeli daha yüksek bir ortalama tahmin sunmaktadır. Varyans açısından XGBoost'un daha yüksek bir varyansa sahip olması, tahminlerinin daha değişken olduğunu gösterir. Bu nedenle, Prophet modeli daha istikrarlı ve güvenilir bir tahmin sunmaktadır.

### 2. EOQ, ROP ve SS Değerleri Karşılaştırması
**Prophet Modeli:**
- EOQ: 380.75
- ROP: 73.90
- SS: 6.78

**XGBoost Modeli:**
- EOQ: 419.76
- ROP: 93.01
- SS: 11.44

**Analiz:**
- XGBoost modeli daha yüksek EOQ, ROP ve SS değerlerine sahiptir. Bu, XGBoost'un daha büyük sipariş miktarları ve daha yüksek güvenlik stokları önerdiği anlamına gelir. Ancak, bu durum daha fazla maliyet anlamına gelebilir. Prophet modeli daha düşük değerler sunarak daha az riskli bir sipariş stratejisi sunmaktadır.

### 3. Toplam Maliyet Analizi
**Prophet Modeli:**
- Toplam Maliyet: 2039.87 TL

**XGBoost Modeli:**
- Toplam Maliyet: 2519.71 TL

**Analiz:**
- Prophet modeli, toplam maliyet açısından daha avantajlıdır. Daha düşük maliyetler, işletmenin karlılığını artırır.

### 4. Varyant Bazlı Risk Skorları
**Yüksek Riskli Varyantlar:**
- Tüm varyantlar (Kırmızı, Mavi, Siyah) için risk skorları 0.855 ile 0.884 arasında değişmektedir. Bu, tüm varyantların yüksek risk taşıdığını göstermektedir.

**Önceliklendirme:**
- Tüm varyantlar yüksek risk taşıdığı için, sipariş önceliği verilmesi gereken varyantlar arasında ayrım yapmak zordur. Ancak, XGBoost modelinin tahminleri daha yüksek olduğu için, bu modelin tahminlerine göre sipariş verilmesi önerilebilir.

### 5. Sipariş Önerisi
- **Toplam Sipariş Miktarı:** 
  - Prophet: 51.90
  - XGBoost: 71.01
- **Öncelik Verilecek Varyantlar:**
  - Tüm varyantlar yüksek risk taşıdığı için, sipariş miktarları eşit dağıtılabilir. Ancak, XGBoost modelinin tahminleri dikkate alınarak, her varyant için önerilen sipariş miktarları artırılabilir.

### 6. Nihai Model Seçimi
**Seçim: Prophet Modeli**
- Prophet modeli, daha düşük maliyetler, daha istikrarlı tahminler ve daha az risk sunmaktadır. Ayrıca, daha düşük EOQ ve ROP değerleri ile daha az sermaye bağlamaktadır.

### 7. Nihai Karar ve Öneri
- **Servis Seviyesi:** %95, bu da müşteri memnuniyetini artırır.
- **Stok-out Riski:** Prophet modeli ile daha düşük.
- **Maliyetler:** Prophet modeli daha düşük maliyet sunuyor.
- **Varyant Dengesi:** Tüm varyantlar yüksek risk taşıyor, bu nedenle eşit dağıtım önerilebilir.
- **Operasyonel Uygulanabilirlik:** Prophet modeli daha az risk ve maliyet sunduğu için daha uygulanabilir.

**Nihai Sipariş Miktarı:** 51 adet (her varyant için eşit dağıtım önerilir).

**Öneri:** Prophet modeline dayanarak, toplam 51 adet sipariş verilmesi ve tüm varyantlar arasında eşit dağıtım yapılması önerilmektedir. Bu, maliyetleri minimize ederken, müşteri memnuniyetini de artıracaktır.